In [12]:
from network import model_builder
from data import *
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [3]:
SHAPE = CROP_SIZE + (1,)

In [13]:
model = model_builder(shape=SHAPE)

In [14]:
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy())

In [15]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_plot2.png', show_shapes=True)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 10, 1)]    0         []                            
                                                                                                  
 conv3d_19 (Conv3D)          (None, 224, 224, 10, 26)     728       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 224, 224, 10, 26)     104       ['conv3d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 leaky_re_lu_18 (LeakyReLU)  (None, 224, 224, 10, 26)     0         ['batch_normalization_18[0

In [6]:
loop = asyncio.get_event_loop()
rows = asyncio.run_coroutine_threadsafe(tensor_train_data(), loop)

-0.8519464753572087
-0.8896387948485123
-1.0280742253090176
-0.9129419797278258
-0.9743737374949517
-0.9281774230450657
-0.9698459529834582
-1.1349136076324802
-0.9445638151349273
-1.1421318483129843
-0.6904739405878332
-0.6916532099471978
-0.9651189155117622
-0.712035951226505
-0.7014618942765081
-0.8817142744334086
-0.8803631051766992
-0.9665412883759924
-1.0349626431818297
-1.0408984777618553
-0.820176600804282
-0.8188317084979493
-0.5231167485943823
-0.5244856236911156
-1.0215196225218186
-1.0504526686418447
-1.040294157743737
-0.9890655623671366
-0.9472611508620196
-0.8997715520283049
-0.9506042759356943
-0.9347231006842435
-0.8301262532297113
-0.6859072678574913
-0.9425274164109978
-0.9617017984023649
-0.7542454945603313
-0.9618232637882429
-0.9452916279739841
-0.7188713160003529
-0.8225024056527255
-0.7575256735517552
-0.7301246400302727
-1.0128905351068291
-0.8531657494671966
-0.8082769797962996
-0.8571666470249732
-0.9308429780456067
-1.1038747538617422
-1.0987396599522115
-1.

In [7]:
result = np.asarray(rows.result())
print(result.shape)
x = result[:,0,:,:,:]
y = result[:,0,:,:,:]

(200, 2, 224, 224, 10)


In [8]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

inputs = np.concatenate((x_train, x_val), axis=0)
targets = np.concatenate((y_train, y_val), axis=0)

In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2319957966061034461
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3760193536
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1695581154270215180
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-03-12 21:58:02.991196: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 21:58:02.991294: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 21:58:02.991330: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 21:58:02.991587: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 21:58:02.991601: I tensorflow/core/common_runtime/gpu/gpu

In [11]:
kf = KFold(n_splits=5, shuffle=True)

fold_scores = []
for train_index, val_index in kf.split(inputs, targets):
    # Train the model on the training data for this fold
    model.fit(inputs[train_index], targets[train_index], epochs=15, batch_size=5, verbose=0)
    
    # Evaluate the model on the validation data for this fold
    _, val = model.evaluate(inputs[val_index], targets[val_index], verbose=0)
    fold_scores.append(val)

average_val = np.mean(fold_scores)
print(average_val)

2024-03-12 21:59:50.495214: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 321126400 exceeds 10% of free system memory.
2024-03-12 22:00:13.338840: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 62.21MiB (rounded to 65228800)requested by op U-Net/max_pooling3d/MaxPool3D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-12 22:00:13.338907: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-03-12 22:00:13.338919: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 144, Chunks in use: 144. 36.0KiB allocated for chunks. 36.0KiB in use in bin. 14.3KiB client-requested in use in bin.
2024-03-12 22:00:13.338925: I external/local_tsl/tsl/framework/bfc_allocator.c

ResourceExhaustedError: Graph execution error:

Detected at node U-Net/max_pooling3d/MaxPool3D defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/ethan/IDC_Group1/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ethan/IDC_Group1/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ethan/IDC_Group1/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ethan/IDC_Group1/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/ethan/IDC_Group1/.venv/lib/